# RAG Utilizzando il Servizio Azure AI Agent e Semantic Kernel

Questo frammento di codice dimostra come creare e gestire un agente Azure AI per la generazione aumentata dal recupero (RAG) utilizzando il `Servizio Azure AI Agent` e `Semantic Kernel`. L'agente elabora le domande degli utenti basandosi sul contesto recuperato e fornisce risposte accurate di conseguenza.


Correzione della versione di SQLite  
Se riscontri l'errore:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Rimuovi il commento da questo blocco di codice all'inizio del tuo notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importazione dei pacchetti
Il seguente codice importa i pacchetti necessari:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generazione con Recupero Aumentato con Semantic Kernel e Azure AI Agent Service

Questo esempio dimostra come utilizzare il **Azure AI Agent Service** per eseguire la **Generazione con Recupero Aumentato (RAG)** combinando un modello linguistico con un contesto specifico del dominio proveniente da un documento caricato.

### Come Funziona

1. **Caricamento del Documento**: Un file markdown (document.md) contenente informazioni (la polizza di assicurazione viaggio di Contoso) viene caricato nel servizio agent.

2. **Creazione dello Store di Vettori**: Il documento viene indicizzato in uno store di vettori per abilitare la ricerca semantica sui suoi contenuti.

3. **Configurazione dell'Agente**: Un agente viene istanziato utilizzando il modello `gpt-4o` con le seguenti istruzioni rigide:
   - Rispondere alle domande solo basandosi sui contenuti recuperati dal documento.
   - Rifiutarsi di rispondere se la domanda è fuori ambito.

4. **Integrazione dello Strumento di Ricerca File**: Il `FileSearchTool` viene registrato con l'agente, permettendo al modello di cercare e recuperare frammenti rilevanti dal documento indicizzato durante l'inferenza.

5. **Interazione con l'Utente**: Gli utenti possono porre domande. Se vengono trovate informazioni pertinenti nel documento, l'agente genera una risposta basata sui dati.  
   In caso contrario, l'agente risponde esplicitamente che il documento non contiene informazioni sufficienti.


### Funzione Principale

La funzione principale è il punto di ingresso del programma. È qui che il codice inizia a essere eseguito.

#### Parametri
- `args`: Un array di stringhe che rappresenta gli argomenti della riga di comando.

#### Esempio
Ecco un esempio di come chiamare la funzione principale:

```python
if __name__ == "__main__":
    main()
```

#### Note
- Assicurati che la funzione principale sia ben strutturata e facile da leggere.
- Evita di inserire troppa logica direttamente nella funzione principale; utilizza funzioni ausiliarie quando possibile.

#### Errori Comuni
- Non gestire correttamente gli argomenti della riga di comando.
- Non includere un controllo `if __name__ == "__main__"`.

#### Suggerimenti
- Utilizza librerie come `argparse` per gestire gli argomenti della riga di comando.
- Scrivi test per verificare che la funzione principale si comporti come previsto.


Assicurati di eseguire prima `az login` utilizzando l'Azure CLI in modo che venga fornito il contesto di autenticazione corretto durante l'utilizzo di `DefaultAzureCredential`. Il servizio Azure AI Agent non utilizza chiavi API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Sure! Please provide the markdown file you'd like me to translate.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
